<a href="https://colab.research.google.com/github/wilzamguerrero/blendz_colab/blob/main/BLNDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend1.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
import os

download_path = "/content/sample_data" #@param {type: "string"}
blender_url = "https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz" #@param {type: "string"}

print('\033[90m' + 'BLENDZ Installing Requirements 1% ✓' + '\033[0m')
with capture.capture_output() as cap:

  os.makedirs(download_path, exist_ok=True)
  !wget {blender_url} -P {download_path}
  os.chdir(download_path)

print('\033[90m' + 'BLENDZ Installing Requirements 30% ✓' + '\033[0m')
with capture.capture_output() as cap:

  !tar xf blender-*.tar.xz
  os.environ["LD_PRELOAD"] = ""
  !apt remove libtcmalloc-minimal4
  !apt install libtcmalloc-minimal4
  os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

print('\033[90m' + 'BLENDZ Installing Requirements 60% ✓' + '\033[0m')
with capture.capture_output() as cap:
  !sudo apt-get purge nvidia*
  !sudo add-apt-repository ppa:graphics-drivers/ppa -y
  !sudo apt-get update
  !sudo apt-get install expect -y

  expect_script = """
  spawn sudo apt install nvidia-driver-510 nvidia-dkms-510 -y
  expect "Configuring keyboard-configuration"
  send "32\r"
  expect "Country of origin for the keyboard"
  send "1\r"
  expect eof
  """
  with open("install_nvidia.exp", "w") as script_file:
      script_file.write(expect_script)

  !expect install_nvidia.exp
  os.remove("install_nvidia.exp")

  clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Installation Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

from IPython.display import clear_output, display, HTML
blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "CYCLES CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 105  # @param {type:"number"}
use_sequence = False  # @param {type:"boolean"}
end_frame = 107  # @param {type:"number"}

image_format = "PNG"  # @param ["PNG", "JPEG", "TIFF"]

output_path = "/content/sample_data"  # @param {type: "string"}

control_samples = True  # @param {type:"boolean"}
samples = 128  # @param {type:"number"}

if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame}"

samples_option = f"-- --cycles-samples {samples}" if control_samples else ""

if engine_option == "CYCLES CUDA":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA {samples_option}
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    !{blender_path}/blender -P "gpu.py" -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX {samples_option}

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Installation Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/install.png?raw=true' height="50" alt="netdata"/></center>
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect,60000)
"""

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/compress.png?raw=true' height="50" alt="netdata"/></center>
import os, sys, re

folder_path = "/content/sample_data" #@param {type:"string"}
archiving_method = 'zip' #@param ["zip", "7z" ]

output_file_path = re.search("^[\/].+\/", folder_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
folder_name = re.sub("^[\/]", "", delsplit.group(0))

os.environ['inputDir'] = folder_path
os.environ['outputPath'] = output_file_path_raw
os.environ['folderName'] = folder_name

if archiving_method == 'zip':
   !7z a -tzip "$outputPath"/"$folderName".zip "$inputDir"

if archiving_method == '7z':
   !7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on "$outputPath"/"$folderName".7z "$inputDir"

In [ ]:
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'CYCLES' --debug-all -o "/content/sample_data/####" -s 105 -e 107 -a -F 'PNG' -- --cycles-device CUDA
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'BLENDER_EEVEE_NEXT' --debug-all -o "/content/sample_data/####" -s 1 -e 20 -a -F 'PNG' -- --BLENDER_EEVEE_NEXT-device CUDA
!/content/sample_data/blender-4.2.0-linux-x64/blender -P 'gpu.py' -b $filename -noaudio -E 'CYCLES' --debug-all -o "/content/sample_data/####" -s 108 -e 110 -a -F 'PNG' -- --cycles-device OPTIX

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "CYCLES CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 109  # @param {type:"number"}
use_sequence = False  # @param {type:"boolean"}
end_frame = 112  # @param {type:"number"}

image_format = "PNG"  # @param ["PNG", "JPEG", "TIFF"]

output_path = "/content/sample_data"  # @param {type: "string"}

control_samples = True  # @param {type:"boolean"}
samples = 10  # @param {type:"number"}

if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame}"

# Crear un script Python para configurar los samples
script_content = f"""
import bpy
bpy.context.scene.cycles.samples = {samples}
"""

with open("set_samples.py", "w") as script_file:
    script_file.write(script_content)

# Construir el comando para ejecutar Blender
if engine_option == "CYCLES CUDA":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX

# Eliminar el script temporal
os.remove("set_samples.py")

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Render Complete 100% ✓' + '\033[0m')


In [ ]:
#@markdown progrss<left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/blend2.png?raw=true' height="50" alt="netdata"/></center>

blender_path = "/content/sample_data/blender-4.2.0-linux-x64"  # @param {type: "string"}
filename = "/content/a2.blend"  # @param {type: "string"}
engine_option = "CYCLES CUDA"  # @param ["CYCLES CUDA", "CYCLES OPTIX", "BLENDER_EEVEE_NEXT CUDA"]

start_frame = 130  # @param {type:"number"}
use_sequence = True  # @param {type:"boolean"}
end_frame = 150  # @param {type:"number"}

image_format = "PNG"  # @param ["PNG", "JPEG", "TIFF"]

output_path = "/content/sample_data"  # @param {type: "string"}

control_samples = True  # @param {type:"boolean"}
samples = 10  # @param {type:"number"}

if use_sequence:
    frame_option = f"-s {start_frame} -e {end_frame}"
else:
    frame_option = f"-s {start_frame} -e {start_frame}"

# Crear un script Python para configurar los samples
script_content = f"""
import bpy
bpy.context.scene.cycles.samples = {samples}
"""

with open("set_samples.py", "w") as script_file:
    script_file.write(script_content)

# Construir el comando para ejecutar Blender
if engine_option == "CYCLES CUDA":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device CUDA
elif engine_option == "BLENDER_EEVEE_NEXT CUDA":
    !{blender_path}/blender -b {filename} -noaudio -E "BLENDER_EEVEE_NEXT" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --BLENDER_EEVEE_NEXT-device CUDA
elif engine_option == "CYCLES OPTIX":
    if control_samples:
        !{blender_path}/blender -b {filename} -P set_samples.py -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX
    else:
        !{blender_path}/blender -b {filename} -noaudio -E "CYCLES" --debug-all -o "{output_path}/####" {frame_option} -a -F "{image_format}" -- --cycles-device OPTIX

# Eliminar el script temporal
os.remove("set_samples.py")

clear_output()
print('\033[38;2;0;243;243m' + 'BLENDZ Render Complete 100% ✓' + '\033[0m')


In [ ]:
!apt install blender
!apt install libboost-all-dev
!apt install libgl1-mesa-dev